In [ ]:
1st varient

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score, recall_score, accuracy_score
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential, clone_model
from tensorflow.keras.layers import Dense
import random
import tensorflow as tf

class IntrusionDetectionEnv:
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.current_idx = 0
        self.state = None

    def reset(self):
        self.current_idx = 0
        self.state = self.X[self.current_idx]
        return self.state

    def step(self, action):
        reward = 1 if action == self.y[self.current_idx] else -1
        self.current_idx += 1
        done = self.current_idx >= len(self.X)
        next_state = self.X[self.current_idx] if not done else None
        return next_state, reward, done

class DDQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = []
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()
        self.target_model = clone_model(self.model)
        self.update_target_model()

    def _build_model(self):
        model = Sequential()
        model.add(Dense(64, input_dim=self.state_size, activation='relu'))
        model.add(Dense(32, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=self.learning_rate), loss='mse')
        return model

    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        q_values = self.model.predict(state, verbose=0)
        return np.argmax(q_values[0])

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = self.model.predict(state, verbose=0)
            if done:
                target[0][action] = reward
            else:
                next_q_values = self.model.predict(next_state, verbose=0)
                next_action = np.argmax(next_q_values[0])
                target_q_values = self.target_model.predict(next_state, verbose=0)
                target[0][action] = reward + self.gamma * target_q_values[0][next_action]
            self.model.fit(state, target, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

def federated_averaging(weights):
    avg_weights = [np.mean([weights[i][j] for i in range(len(weights))], axis=0) for j in range(len(weights[0]))]
    return avg_weights

data = pd.read_csv("TON_IoT.csv")
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['label'])

X = data.drop('label', axis=1).values
y = data['label'].values

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_resampled)

pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_scaled)

X_train, X_test, y_train, y_test = train_test_split(X_pca, y_resampled, test_size=0.2, random_state=42)

num_clients = 10
X_partitions = np.array_split(X_train, num_clients)
y_partitions = np.array_split(y_train, num_clients)

state_size = X_train.shape[1]
action_size = len(np.unique(y))
global_agent = DDQNAgent(state_size, action_size)
global_weights = global_agent.model.get_weights()

clients = [DDQNAgent(state_size, action_size) for _ in range(num_clients)]
num_rounds = 10
batch_size = 32

results_df = pd.DataFrame(columns=["Round", "Q-value (AVG)", "F1-score", "Recall", "Acc (%)", "Cumulative Reward (AVG)"])

for round_num in range(num_rounds):
    local_weights = []
    cumulative_rewards = []
    q_values = []

    for client_idx, client in enumerate(clients):
        X_client = X_partitions[client_idx]
        y_client = y_partitions[client_idx]

        env = IntrusionDetectionEnv(X_client, y_client)
        client.model.set_weights(global_weights)
        client.update_target_model()

        state = env.reset().reshape(1, state_size)
        total_reward = 0
        total_q_value = 0
        steps = 0

        while True:
            action = client.act(state)
            next_state, reward, done = env.step(action)
            next_state = next_state.reshape(1, state_size) if next_state is not None else None
            client.remember(state, action, reward, next_state, done)
            state = next_state
            total_reward += reward

            q_value = np.max(client.model.predict(state, verbose=0)) if state is not None else 0
            total_q_value += q_value
            steps += 1

            if done:
                break
            if len(client.memory) > batch_size:
                client.replay(batch_size)

        cumulative_rewards.append(total_reward / steps)
        q_values.append(total_q_value / steps)
        local_weights.append(client.model.get_weights())

    global_weights = federated_averaging(local_weights)
    global_agent.model.set_weights(global_weights)
    global_agent.update_target_model()

    y_pred = np.argmax(global_agent.model.predict(X_test, verbose=0), axis=1)
    f1 = f1_score(y_test, y_pred, average="weighted")
    recall = recall_score(y_test, y_pred, average="weighted")
    acc = accuracy_score(y_test, y_pred) * 100

    results_df.loc[round_num] = [
        round_num + 1,
        np.mean(q_values),
        f1 * 100,
        recall * 100,
        acc,
        np.mean(cumulative_rewards),
    ]

    print(f"Round {round_num + 1}/{num_rounds} complete.")

print(results_df)
results_df.to_csv("training_results_ddqn.csv", index=False)


In [ ]:
2nd-varient

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score, recall_score, accuracy_score
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential, clone_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
import random
import tensorflow as tf

class IntrusionDetectionEnv:
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.current_idx = 0
        self.state = None

    def reset(self):
        self.current_idx = 0
        self.state = self.X[self.current_idx]
        return self.state

    def step(self, action):
        reward = 1 if action == self.y[self.current_idx] else -1
        self.current_idx += 1
        done = self.current_idx >= len(self.X)
        next_state = self.X[self.current_idx] if not done else None
        return next_state, reward, done

class AdvancedDDQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = []
        self.gamma = 0.99
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.990
        self.learning_rate = 0.0005
        self.model = self._build_model()
        self.target_model = clone_model(self.model)
        self.update_target_model()

    def _build_model(self):
        model = Sequential()
        model.add(Dense(128, input_dim=self.state_size, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=self.learning_rate), loss='huber')
        return model

    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        q_values = self.model.predict(state, verbose=0)
        return np.argmax(q_values[0])

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = self.model.predict(state, verbose=0)
            if done:
                target[0][action] = reward
            else:
                next_q = self.model.predict(next_state, verbose=0)
                best_action = np.argmax(next_q[0])
                target_q = self.target_model.predict(next_state, verbose=0)
                target[0][action] = reward + self.gamma * target_q[0][best_action]
            self.model.fit(state, target, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

def federated_averaging(weights):
    return [np.mean([client_weights[i] for client_weights in weights], axis=0) for i in range(len(weights[0]))]

data = pd.read_csv("TON_IoT.csv")
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['label'])

X = data.drop('label', axis=1).values
y = data['label'].values

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_resampled)

pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_scaled)

X_train, X_test, y_train, y_test = train_test_split(X_pca, y_resampled, test_size=0.2, random_state=42)

num_clients = 10
X_partitions = np.array_split(X_train, num_clients)
y_partitions = np.array_split(y_train, num_clients)

state_size = X_train.shape[1]
action_size = len(np.unique(y))
global_agent = AdvancedDDQNAgent(state_size, action_size)
global_weights = global_agent.model.get_weights()

clients = [AdvancedDDQNAgent(state_size, action_size) for _ in range(num_clients)]
num_rounds = 10
batch_size = 64

results_df = pd.DataFrame(columns=["Round", "Q-value (AVG)", "F1-score", "Recall", "Acc (%)", "Cumulative Reward (AVG)"])

for round_num in range(num_rounds):
    local_weights = []
    cumulative_rewards = []
    q_values = []

    for client_idx, client in enumerate(clients):
        X_client = X_partitions[client_idx]
        y_client = y_partitions[client_idx]

        env = IntrusionDetectionEnv(X_client, y_client)
        client.model.set_weights(global_weights)
        client.update_target_model()

        state = env.reset().reshape(1, state_size)
        total_reward = 0
        total_q_value = 0
        steps = 0

        while True:
            action = client.act(state)
            next_state, reward, done = env.step(action)
            next_state = next_state.reshape(1, state_size) if next_state is not None else None
            client.remember(state, action, reward, next_state, done)
            state = next_state
            total_reward += reward
            q_value = np.max(client.model.predict(state, verbose=0)) if state is not None else 0
            total_q_value += q_value
            steps += 1
            if done:
                break
            if len(client.memory) > batch_size:
                client.replay(batch_size)

        cumulative_rewards.append(total_reward / steps)
        q_values.append(total_q_value / steps)
        local_weights.append(client.model.get_weights())

    global_weights = federated_averaging(local_weights)
    global_agent.model.set_weights(global_weights)
    global_agent.update_target_model()

    y_pred = np.argmax(global_agent.model.predict(X_test, verbose=0), axis=1)
    f1 = f1_score(y_test, y_pred, average="weighted")
    recall = recall_score(y_test, y_pred, average="weighted")
    acc = accuracy_score(y_test, y_pred) * 100

    results_df.loc[round_num] = [
        round_num + 1,
        np.mean(q_values),
        f1 * 100,
        recall * 100,
        acc,
        np.mean(cumulative_rewards),
    ]

    print(f"Round {round_num + 1}/{num_rounds} complete.")

print(results_df)
results_df.to_csv("advanced_ddqn_results.csv", index=False)
